In [37]:
import sys, os, numpy as np, pytest

sys.path.append(os.path.abspath("../src"))

from pricing_black_scholes import black_scholes
from pricing_binomial import binomial_american
# from pricing_binomial import binomial_european
from pricing_montecarlo import monte_carlo_asian
from pricing_montecarlo import monte_carlo_digital_barrier
from pricing_montecarlo import monte_carlo_lookback

#### Princign black scholes

In [38]:
def test_call_option_price():
    # Ejemplo clásico Black–Scholes
    price = black_scholes(S=100, K=100, T=1, r=0.05, sigma=0.2, option_type="call")
    assert round(price, 2) == 10.45


def test_put_option_price():
    price = black_scholes(S=100, K=100, T=1, r=0.05, sigma=0.2, option_type="put")
    assert round(price, 2) == 5.57


def test_invalid_option_type():
    with pytest.raises(ValueError):
        black_scholes(S=100, K=100, T=1, r=0.05, sigma=0.2, option_type="invalid")

tests = [
    ("call",  test_call_option_price),
    ("put",   test_put_option_price),
    ("error", test_invalid_option_type),
]

for name, fn in tests:
    try:
        fn()                         # Execute the test function
        print(f"{name}: ✅ OK")
    except AssertionError as e:
        print(f"{name}: ❌ {e}")

call: ✅ OK
put: ✅ OK
error: ✅ OK


#### Princing Binomial

In [39]:
def print_test(title, passed):
    status = "✅ PASSED" if passed else "❌ FAILED"
    print(f"{title.ljust(55)} {status}")


def test_binomial_vs_bs(option_type):
    S, K, T, r, sigma, N = 100, 100, 1.0, 0.05, 0.2, 1000
    bin_price = binomial_american(S, K, T, r, sigma, N, option_type)
    bs_price = black_scholes(S, K, T, r, sigma, option_type)
    error = abs(bin_price - bs_price)

    print(f"{option_type.capitalize()} Option:")
    print(f"  Binomial: {bin_price:.4f},  BS: {bs_price:.4f},  Abs Error: {error:.4f}")
    print_test(f"Binomial ≈ Black-Scholes ({option_type})", error < 1e-2)


def test_american_put_greater_than_bs():
    S, K, T, r, sigma, N = 90, 100, 1.0, 0.03, 0.25, 100
    bin_price = binomial_american(S, K, T, r, sigma, N, 'put')
    bs_price = black_scholes(S, K, T, r, sigma, 'put')

    print("Put Option (In the Money):")
    print(f"  Binomial: {bin_price:.4f},  BS: {bs_price:.4f}")
    print_test("American put ≥ European put", bin_price >= bs_price)


def test_invalid_option_type():
    try:
        binomial_american(100, 100, 1.0, 0.05, 0.2, 100, option_type='banana')
        print_test("Invalid option type raises error", False)
    except ValueError:
        print_test("Invalid option type raises error", True)


def test_negative_inputs():
    test_cases = [
        (-100, 100, 1.0, 0.05, 0.2),
        (100, 0, 1.0, 0.05, 0.2),
        (100, 100, -1.0, 0.05, 0.2),
        (100, 100, 1.0, 0.05, -0.2),
    ]
    for i, (S, K, T, r, sigma) in enumerate(test_cases, 1):
        try:
            binomial_american(S, K, T, r, sigma, 100, option_type='call')
            print_test(f"Negative input test #{i}", False)
        except ValueError:
            print_test(f"Negative input test #{i}", True)


if __name__ == "__main__":
    print("\nRunning manual tests for binomial_american()\n" + "-" * 60)
    test_binomial_vs_bs('call')
    test_binomial_vs_bs('put')
    test_american_put_greater_than_bs()
    test_invalid_option_type()
    test_negative_inputs()



Running manual tests for binomial_american()
------------------------------------------------------------
Call Option:
  Binomial: 10.4486,  BS: 10.4506,  Abs Error: 0.0020
Binomial ≈ Black-Scholes (call)                         ✅ PASSED
Put Option:
  Binomial: 6.0896,  BS: 5.5735,  Abs Error: 0.5161
Binomial ≈ Black-Scholes (put)                          ❌ FAILED
Put Option (In the Money):
  Binomial: 13.7986,  BS: 13.2426
American put ≥ European put                             ✅ PASSED
Invalid option type raises error                        ✅ PASSED
Negative input test #1                                  ✅ PASSED
Negative input test #2                                  ✅ PASSED
Negative input test #3                                  ✅ PASSED
Negative input test #4                                  ✅ PASSED


In [40]:
def test_binomial_european_vs_bs():
    S, K, T, r, sigma, N = 100, 100, 1.0, 0.05, 0.2, 1000
    for option_type in ["call", "put"]:
        bin_price = binomial_european(S, K, T, r, sigma, N, option_type)
        bs_price = black_scholes(S, K, T, r, sigma, option_type)
        error = abs(bin_price - bs_price)
        print(f"{option_type.capitalize()} Option (European):")
        print(f"  Binomial: {bin_price:.4f},  BS: {bs_price:.4f},  Abs Error: {error:.4f}")
        print_test(f"European binomial ≈ BS ({option_type})", error < 1e-2)


if __name__ == "__main__":
    print("\nRunning tests for binomial_european()\n" + "-" * 55)
    test_binomial_european_vs_bs()



Running tests for binomial_european()
-------------------------------------------------------


NameError: name 'binomial_european' is not defined

#### Princing Montecarlo Asian

In [ ]:
def print_test(title, passed):
    status = "✅ PASSED" if passed else "❌ FAILED"
    print(f"{title.ljust(50)} {status}")


def test_mc_value_repeatable():
    np.random.seed(42)
    price = monte_carlo_asian(S=100, K=100, T=1.0, r=0.05, sigma=0.2,
                                       option_type='call', n_simulations=5000, n_steps=50)
    expected = 5.9  # Aproximado con semilla fija
    passed = abs(price - expected) < 0.5
    print(f"MC Asian Call Price: {price:.4f} (expected ≈ {expected})")
    print_test("Monte Carlo reproducible result", passed)


def test_mc_put_greater_when_ITM():
    np.random.seed(0)
    call_price = monte_carlo_asian(80, 100, 1.0, 0.05, 0.3, 'call')
    put_price = monte_carlo_asian(80, 100, 1.0, 0.05, 0.3, 'put')
    print_test("Put price > Call price when ITM for put", put_price > call_price)


def test_invalid_option_type():
    try:
        monte_carlo_asian(100, 100, 1.0, 0.05, 0.2, option_type='banana')
        print_test("Invalid option type raises error", False)
    except ValueError:
        print_test("Invalid option type raises error", True)


def test_negative_inputs():
    try:
        monte_carlo_asian(-100, 100, 1.0, 0.05, 0.2)
        print_test("Negative input check", False)
    except ValueError:
        print_test("Negative input check", True)


if __name__ == "__main__":
    print("\nRunning tests for monte_carlo_asian()\n" + "-"*55)
    test_mc_value_repeatable()
    test_mc_put_greater_when_ITM()
    test_invalid_option_type()
    test_negative_inputs()



Running tests for monte_carlo_asian()
-------------------------------------------------------
MC Asian Call Price: 5.9365 (expected ≈ 5.9)
Monte Carlo reproducible result                    ✅ PASSED
Put price > Call price when ITM for put            ✅ PASSED
Invalid option type raises error                   ✅ PASSED
Negative input check                               ✅ PASSED


#### Monte Carlo Digital Barrier

In [ ]:
def test_digital_barrier_price_valid():
    price = monte_carlo_digital_barrier(
        S=100, K=100, T=1, r=0.05, sigma=0.2,
        barrier=110, option_type='call',
        barrier_type='up-and-in',
        n_simulations=5000, n_steps=50
    )
    assert 0 <= price <= 1, f"Invalid price: {price}"
    print(f"✅ Digital up-and-in call price: {price:.4f}")

def test_barrier_type_difference():
    price_in = monte_carlo_digital_barrier(
        S=100, K=100, T=1, r=0.05, sigma=0.2,
        barrier=110, option_type='call',
        barrier_type='up-and-in',
        n_simulations=5000, n_steps=50
    )
    price_out = monte_carlo_digital_barrier(
        S=100, K=100, T=1, r=0.05, sigma=0.2,
        barrier=110, option_type='call',
        barrier_type='up-and-out',
        n_simulations=5000, n_steps=50
    )
    assert abs(price_in - price_out) > 0.01, "In and Out prices too close"
    print(f"✅ Barrier type affects price: In={price_in:.4f}, Out={price_out:.4f}")

def test_invalid_barrier_type():
    try:
        monte_carlo_digital_barrier(
            S=100, K=100, T=1, r=0.05, sigma=0.2,
            barrier=110, option_type='call',
            barrier_type='sideways-in'
        )
        assert False, "Invalid barrier type did not raise error"
    except ValueError:
        print("✅ Invalid barrier type raises ValueError")

def test_invalid_option_type():
    try:
        monte_carlo_digital_barrier(
            S=100, K=100, T=1, r=0.05, sigma=0.2,
            barrier=110, option_type='banana',
            barrier_type='up-and-in'
        )
        assert False, "Invalid option type did not raise error"
    except ValueError:
        print("✅ Invalid option type raises ValueError")

if __name__ == "__main__":
    print("\nRunning digital barrier option tests\n" + "-" * 45)
    test_digital_barrier_price_valid()
    test_barrier_type_difference()
    test_invalid_barrier_type()
    test_invalid_option_type()



Running digital barrier option tests
---------------------------------------------
✅ Digital up-and-in call price: 0.4737
✅ Barrier type affects price: In=0.4671, Out=0.0565
✅ Invalid barrier type raises ValueError
✅ Invalid option type raises ValueError


#### Monte Carlo Lookback

In [ ]:
def print_test(title, passed):
    status = "✅ PASSED" if passed else "❌ FAILED"
    print(f"{title.ljust(50)} {status}")

def test_lookback_option_types():
    S, K, T, r, sigma = 100, 100, 1.0, 0.05, 0.2
    call_price = monte_carlo_lookback(S, K, T, r, sigma, option_type='call', n_simulations=10000, n_steps=100)
    put_price = monte_carlo_lookback(S, K, T, r, sigma, option_type='put', n_simulations=10000, n_steps=100)
    print(f"Lookback Call Price: {call_price:.4f}")
    print(f"Lookback Put Price:  {put_price:.4f}")
    print_test("Lookback call and put prices are positive", call_price > 0 and put_price > 0)

def test_invalid_option_type_lookback():
    try:
        monte_carlo_lookback(100, 100, 1.0, 0.05, 0.2, option_type='banana')
        print_test("Invalid option type raises error", False)
    except ValueError:
        print_test("Invalid option type raises error", True)

def test_negative_inputs_lookback():
    test_cases = [
        (-100, 100, 1.0, 0.05, 0.2),
        (100, 0, 1.0, 0.05, 0.2),
        (100, 100, -1.0, 0.05, 0.2),
        (100, 100, 1.0, 0.05, -0.2),
    ]
    for i, (S, K, T, r, sigma) in enumerate(test_cases, 1):
        try:
            monte_carlo_lookback(S, K, T, r, sigma, option_type='call')
            print_test(f"Negative input test #{i}", False)
        except ValueError:
            print_test(f"Negative input test #{i}", True)

if __name__ == "__main__":
    print("\nRunning tests for monte_carlo_lookback()\n" + "-" * 55)
    test_lookback_option_types()
    test_invalid_option_type_lookback()
    test_negative_inputs_lookback()



Running tests for monte_carlo_lookback()
-------------------------------------------------------
Lookback Call Price: 17.8050
Lookback Put Price:  11.4158
Lookback call and put prices are positive          ✅ PASSED
Invalid option type raises error                   ✅ PASSED
Negative input test #1                             ✅ PASSED
Negative input test #2                             ✅ PASSED
Negative input test #3                             ✅ PASSED
Negative input test #4                             ✅ PASSED
